In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import yfinance as yf

#dfSPY=yf.download("^GSPC",start='2011-01-05', end='2021-01-05')
dfSPY=yf.download("XLP",start='2002-01-05', end='2023-07-07')
#dfSPY=yf.download("AMZN",start='2011-01-05', end='2021-01-05')

dfSPY=dfSPY[dfSPY.High!=dfSPY.Low]
dfSPY.reset_index(inplace=True)
dfSPY.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-01-07,25.000000,25.129999,24.860001,24.969999,14.403708,173700
1,2002-01-08,24.820000,24.980000,24.799999,24.840000,14.328714,66000
2,2002-01-09,24.900000,25.180000,24.840000,24.930000,14.380638,528500
3,2002-01-10,24.900000,25.080000,24.809999,24.969999,14.403708,41100
4,2002-01-11,25.049999,25.270000,25.010000,25.030001,14.438316,41100


In [3]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas_ta as ta
dfSPY['EMA']=ta.ema(dfSPY.Close, length=233)#sma ema
dfSPY['RSI']=ta.rsi(dfSPY.Close, length=2)
#a=ta.adx(dfSPY.High, dfSPY.Low, dfSPY.Close, length=14)
#dfSPY=dfSPY.join(a.ADX_14)

#dfSPY.ta.indicators()
#help(ta.adx)
dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)
dfSPY[420:425]

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,RSI
420,652,2004-08-10,21.670000,21.860001,21.670000,21.850000,13.153989,231000,22.186503,67.560211
421,653,2004-08-11,21.799999,22.040001,21.750000,22.020000,13.256330,201400,22.185080,84.249354
422,654,2004-08-12,22.139999,22.150000,21.969999,21.969999,13.226233,229300,22.183242,64.676182
423,655,2004-08-13,22.080000,22.110001,21.959999,22.070000,13.286433,1228200,22.182274,81.690668
424,656,2004-08-16,22.120001,22.209999,22.010000,22.209999,13.370708,707300,22.182511,92.204383


In [5]:
def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.EMA[i]:
                dnt=0
            if df.Low[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal

addemasignal(dfSPY,6)

In [6]:
def totalSignal(df):
    ordersignal=[0]*len(df)
    for i in range(0, len(df)):
        if  df.RSI[i]<=5  and df.EMASignal[i]==2:
            ordersignal[i]=2
    df['ordersignal']=ordersignal

totalSignal(dfSPY)

In [7]:
dfSPY[dfSPY.ordersignal!=0]

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,RSI,EMASignal,ordersignal
220,452,2003-10-22,20.950001,21.040001,20.830000,20.930000,12.455013,274600,20.558123,4.943121,2,2
221,453,2003-10-23,20.799999,20.969999,20.799999,20.930000,12.455013,289000,20.561301,4.943121,2,2
222,454,2003-10-24,20.920000,20.920000,20.629999,20.910000,12.443112,289900,20.564281,3.138757,2,2
223,455,2003-10-27,21.030001,21.049999,20.830000,20.900000,12.437155,166600,20.567151,2.299413,2,2
316,548,2004-03-11,22.879999,22.980000,22.570000,22.620001,13.517728,886700,21.335303,0.785407,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
5043,5275,2022-12-19,74.309998,74.940002,73.839996,74.279999,71.832352,8093900,73.134101,2.606047,2,2
5044,5276,2022-12-20,74.150002,74.510002,73.809998,74.220001,71.774330,7368900,73.143382,2.358674,2,2
5146,5378,2023-05-18,75.959999,76.059998,75.410004,75.870003,73.722679,10765300,73.947786,4.497993,2,2
5148,5380,2023-05-22,75.720001,75.849998,74.730003,74.769997,72.653809,14575700,73.971526,2.399326,2,2


# Visualization

In [8]:
import numpy as np
def pointpos(x):
    if x['ordersignal']==1:
        return x['High']+2e-3
    elif x['ordersignal']==2:
        return x['Low']-2e-3
    else:
        return np.nan

dfSPY['pointpos'] = dfSPY.apply(lambda row: pointpos(row), axis=1)

In [9]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install nbformat --upgrade


Note: you may need to restart the kernel to use updated packages.


In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = dfSPY[1000:3200].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=6, color="MediumPurple"),
                name="Signal")
#fig.update(layout_yaxis_range = [300,420])
fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(autosize=False, width=1000, height=600,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

In [12]:
dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [13]:
pip install backtesting

Note: you may need to restart the kernel to use updated packages.


In [14]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        perc=0.02
        if len(self.trades)>0:
            if self.data.index[-1]-self.trades[-1].entry_time>=1000:
                self.trades[-1].close()
            elif self.trades[-1].is_long and self.data.RSI[-1]>=75:
                self.trades[-1].close()
            #elif self.trades[-1].is_short and self.data.RSI[-1]<=20:
           #     self.trades[-1].close()

        if self.signal==2 and len(self.trades)==0:
            sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1-perc)
            #tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            #self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1,size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1, commission=.000)
stat = bt.run()
stat

/tmp/ipykernel_3335720/3975604623.py:29: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    5178.0
Duration                               5178.0
Exposure Time [%]                    8.920641
Equity Final [$]                 13304.855778
Equity Peak [$]                  13683.674522
Return [%]                          33.048558
Buy & Hold Return [%]              268.283046
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -6.195481
Avg. Drawdown [%]                    -1.63424
Max. Drawdown Duration                  579.0
Avg. Drawdown Duration                  118.0
# Trades                                 87.0
Win Rate [%]                        71.264368
Best Trade [%]                       3.705488
Worst Trade [%]                     -2.799152
Avg. Trade [%]                    

In [15]:
bt.plot()

GridPlot(id='p1307', ...)